# Revert to PorePy setup directly

To better understand potential errors in scaling, we revert to the PorePy setup for scaling by subclassing `ContactMechanicsBiot` with basic and transparent parameter setup. Then, we will gradually make the model more complex, hopefully finding out where `ContactMechanicsBiotISC` breaks.

In [1]:
import os
import sys
from pathlib import Path

import pendulum
import numpy as np
import porepy as pp
from porepy.models.contact_mechanics_biot_model import ContactMechanicsBiot

import logging
logging.basicConfig(level='INFO')

/home/haakon/porepy/src/porepy/viz/exporter.py:928: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  angle = np.arccos(np.dot(normals[:, fi], reference))


In [2]:
_ROOT = '~/mastersproject/src/mastersproject/GTS/test'
logger = logging.getLogger('GTS')

In [31]:
class SimpleContactMechanicsBiot(ContactMechanicsBiot):
    def __init__(self, params):
        """ 
        
        params needs the following keywords
            length_scale, scalar_scale : float
        """
        
        # Back-end
        _root = _ROOT
        name = "test_parameter_scaling_basic_setup_on_regular_grid_ipynb"
        date = pendulum.now().format("YYMMDD")
        _f = f"{name}/{date}/test_1"
        path_head = f"{_f}/ss{params['scalar_scale']}ls{params['length_scale']}"
        _results_path = f"{_root}/results/{_root}"
        Path(_results_path).mkdir(parents=True, exist_ok=True)
        params["folder_name"] = _results_path
        
        super().__init__(params)
        
        self.length_scale = params["length_scale"]
        self.scalar_scale = params["scalar_scale"]
        
        # Time parameters
        self.time = 0
        self.time_step = 1
        self.end_time = 1
        
        
        self.file_name = 'main_run'
        
        # -- Physical parameters --
        # STRESS
        # Create a simple stress tensor, rotated by 45 degrees (to induce slip)
        stress = - np.array([  # Anisotropic stress (negative for compressive)
            [0.5, 0, 0],
            [0, 1.5, 0],
            [0, 0, 1.0]
        ]) # Unscaled stress. Units: [Pa]
        a = np.pi / 4  # Rotate the stress 45 degrees
        vec = np.array([0, 0, 1]) # about the unit vector e_z.
        R = pp.geometry.map_geometry.rotation_matrix(a, vec)
        self.stress = R.dot(stress.dot(R.T)) # Stress in the rotated frame.
        
    def create_grid(self):
        # Size of domain - scaled by length scale.
        physdims = np.array([10, 10, 10]) * pp.METER / self.length_scale
        bounding_box = {
            "xmin": 0, "xmax": physdims[0],
            "ymin": 0, "ymax": physdims[1],
            "zmin": 0, "zmax": physdims[2],
        }
        self.box = bounding_box
        
        f_1 = np.array([
            [5, 5, 5, 5],
            [0, 0, 5, 5],
            [0, 5, 5, 0]
        ]) * pp.METER / self.length_scale
        
        # fully immerse fracture # TEST: A1: double
        f_2 = np.array([
            [5, 5, 5, 5],
            [2.5, 2.5, 5, 5],
            [2.5, 5, 5, 2.5]
        ]) * pp.METER / self.length_scale
        
        # Number of cells in each dimension.
        n_cells = np.array([2, 2*2, 2*2]) # TEST: A1: double
        
        gb = pp.meshing.cart_grid(
            fracs=[f_2],  #[f_1], # TEST: A1: double
            nx=n_cells,
            physdims=physdims,
        )
        self.gb = gb
        pp.contact_conditions.set_projections(self.gb)
        self.Nd = gb.dim_max()
        self.n_frac = gb.get_grids(lambda _g: _g.dim == self.Nd - 1).size
        
    def bc_values_mechanics(self, g):
        """ Set top and sides to neumann stress tensor.
        Clamp the bottom to zero dirichlet
        """
        
        # Retrieve the domain boundary
        all_bf, east, west, north, south, top, bottom = self.domain_boundary_sides(g)
        dim = g.dim

        # Boundary values
        bc_values = np.zeros((g.dim, g.num_faces))

        # --- mechanical state ---
        # Get outward facing normal vectors for domain boundary, weighted for face area

        # 1. Get normal vectors on the faces. These are already weighed by face area.
        bf_normals = g.face_normals * pp.METER**(dim-1)
        # 2. Adjust direction so they face outwards
        flip_normal_to_outwards = np.where(g.cell_face_as_dense()[0, :] >= 0, 1, -1)
        outward_normals = bf_normals * flip_normal_to_outwards
        bf_stress = np.dot(self.stress, outward_normals[:, all_bf])
        # Units [P L^(dim-1)] -- g.face_normals is scaled. Now, scale pressure:
        bc_values[:, all_bf] += bf_stress * pp.PASCAL / self.scalar_scale  # Mechanical stress
        
        # Dirichlet
        # Units: [L]
        bc_values[:, bottom] = 0 * pp.METER  # / self.length_scale
        
        return bc_values.ravel("F")
    
    def bc_type_mechanics(self, g):
        """ Neumann eveywhere except bottom face, where it is Dirichlet"""
        all_bf, east, west, north, south, top, bottom = self.domain_boundary_sides(g)
        bc = pp.BoundaryConditionVectorial(g, bottom, "dir")
        fracture_faces = g.tags["fracture_faces"]
        bc.is_neu[:, fracture_faces] = False
        bc.is_dir[:, fracture_faces] = True
        
        return bc
    
    def bc_values_scalar(self, g):
        """ Dirichlet everywhere. Fluid injection on west face"""
        all_bf, east, west, north, south, top, bottom = self.domain_boundary_sides(g)
        
        bc_values = np.zeros(g.num_faces)
        
        pressure_at_west = 5 * pp.PASCAL / self.scalar_scale
        bc_values[west] = pressure_at_west
        
        return bc_values
        
    def bc_type_scalar(self, g):
        """ Dirichlet everywhere"""
        all_bf, *_ = self.domain_boundary_sides(g)
        # Define boundary condition on faces
        return pp.BoundaryCondition(g, all_bf, ["dir"] * all_bf.size)
    
    def source_scalar(self, g):
        source = np.zeros(g.num_cells)
#         if g.dim == self.Nd-1:  # in a fracture

        # Set in-flow in fracture and on grid.
#         flow_rate = 10 * pp.MILLI * (pp.METER / self.length_scale) ** self.Nd
#         source[0] += flow_rate * self.time_step
        
        return source
    
    def source_mechanics(self, g):
        return np.zeros(self.Nd * g.num_cells)
    
    def _set_friction_coefficient(self, g):
        return np.ones(g.num_cells) * 0.5
    
    def compute_aperture(self, g):
        apertures = np.ones(g.num_cells)
        if g.dim < self.Nd:
            apertures *= 0.1
        return apertures * pp.METER / self.length_scale
    
    def biot_alpha(self, g):
        return 1
    
    def check_convergence(self, solution, prev_solution, init_solution, nl_params=None):
        g_max = self._nd_grid()

        if not self._is_nonlinear_problem():
            # At least for the default direct solver, scipy.sparse.linalg.spsolve, no
            # error (but a warning) is raised for singular matrices, but a nan solution
            # is returned. We check for this.
            diverged = np.any(np.isnan(solution))
            converged = not diverged
            error = np.nan if diverged else 0
            return error, converged, diverged

        mech_dof = self.assembler.dof_ind(g_max, self.displacement_variable)
        scalar_dof = self.assembler.dof_ind(g_max, self.scalar_variable)

        # Also find indices for the contact variables
        contact_dof = np.array([], dtype=np.int)
        for e, _ in self.gb.edges():
            if e[0].dim == self.Nd:
                contact_dof = np.hstack(
                    (
                        contact_dof,
                        self.assembler.dof_ind(e[1], self.contact_traction_variable),
                    )
                )

        # Pick out the solution from current, previous iterates, as well as the
        # initial guess.
        u_mech_now = solution[mech_dof] * self.length_scale
        u_mech_prev = prev_solution[mech_dof] * self.length_scale
        u_mech_init = init_solution[mech_dof] * self.length_scale

        # TODO: Check if scaling of contact variable
        contact_now = solution[contact_dof] * self.scalar_scale * self.length_scale ** 2
        contact_prev = prev_solution[contact_dof] * self.scalar_scale * self.length_scale ** 2
        contact_init = init_solution[contact_dof] * self.scalar_scale * self.length_scale ** 2

        # Pressure solution
        p_scalar_now = solution[scalar_dof] * self.scalar_scale
        p_scalar_prev = prev_solution[scalar_dof] * self.scalar_scale
        p_scalar_init = init_solution[scalar_dof] * self.scalar_scale

        # Calculate errors

        # Displacement error
        difference_in_iterates_mech = np.sum((u_mech_now - u_mech_prev) ** 2)
        difference_from_init_mech = np.sum((u_mech_now - u_mech_init) ** 2)

        logger.info(f"diff iter u = {difference_in_iterates_mech:.6e}")
        logger.info(f"diff init u = {difference_from_init_mech:.6e}")

        # Contact traction error
        # TODO: Unsure about units of contact traction
        contact_norm = np.sum(contact_now ** 2)
        difference_in_iterates_contact = np.sum((contact_now - contact_prev) ** 2)
        difference_from_init_contact = np.sum((contact_now - contact_init) ** 2)

        logger.info(f"diff iter contact = {difference_in_iterates_contact:.6e}")
        logger.info(f"diff init contact = {difference_from_init_contact:.6e}")

        # Pressure scalar error
        scalar_norm = np.sum(p_scalar_now ** 2)
        difference_in_iterates_scalar = np.sum((p_scalar_now - p_scalar_prev) ** 2)
        difference_from_init_scalar = np.sum((p_scalar_now - p_scalar_init) ** 2)
        logger.info(f"diff iter scalar = {difference_in_iterates_scalar:.6e}")
        logger.info(f"diff init scalar = {difference_from_init_scalar:.6e}")

        tol_convergence = nl_params["nl_convergence_tol"]
        # Not sure how to use the divergence criterion
        # tol_divergence = nl_params["nl_divergence_tol"]

        converged = False
        diverged = False

        # Converge in displacement and pressure on 3D grid
        converged_u = False
        converged_p = False

        # Check absolute convergence criterion
        if difference_in_iterates_mech < tol_convergence:
            # converged = True
            converged_u = True
            error_mech = difference_in_iterates_mech
            logger.info(f"u converged absolutely.")

        else:
            # Check relative convergence criterion
            if (
                difference_in_iterates_mech
                < tol_convergence * difference_from_init_mech
            ):
                # converged = True
                converged_u = True
                logger.info(f"u converged relatively")
            error_mech = difference_in_iterates_mech / difference_from_init_mech

        # The if is intended to avoid division through zero
        if difference_in_iterates_contact < tol_convergence:
            # converged = True
            error_contact = difference_in_iterates_contact
            logger.info(f"contact variable converged absolutely")
        else:
            error_contact = (
                difference_in_iterates_contact / difference_from_init_contact
            )

        # -- Scalar solution --
        # The if is intended to avoid division through zero
        if difference_in_iterates_scalar < tol_convergence:
            converged_p = True
            error_scalar = difference_in_iterates_scalar
            logger.info(f"pressure converged absolutely")
        else:
            # Relative convergence criterion:
            if difference_in_iterates_scalar < tol_convergence * difference_from_init_scalar:
                # converged = True
                converged_p = True
                logger.info(f"pressure converged relatively")

            error_scalar = (difference_in_iterates_scalar / difference_from_init_scalar)

        logger.info(f"Error in contact force is {error_contact:.6e}")
        logger.info(f"Error in matrix displacement is {error_mech:.6e}")
        logger.info(f"Error in pressure is {error_scalar:.6e}.")

        converged = converged_p and converged_u

        return error_mech, converged, diverged
    
    

# Check scaling of parameters and solution

In [4]:
def check_parameter_scaling(s1, s2, dim):
    """ Test parameter scaling for two setups"""
    print(f"Checking parameters for two setups on grid of dimension: {dim}")
    
    def fetch_values_from_setup(s, dim):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        g = gb.grids_of_dimension(dim)[0]
        # data
        data = gb.node_props(g)
        params_u = data['parameters']['mechanics']
        params_p = data['parameters']['flow']
        
        neu_p = params_p['bc'].is_neu
        dir_p = params_p['bc'].is_dir
        
        # Compute unscaled quantities: 
        # MECHANICS
        if dim == s.Nd: # On rock matrix
            neu_u = (params_u['bc'].is_neu).ravel(order='F')
            dir_u = (params_u['bc'].is_dir).ravel(order='F')
            mech = {
                'bc_values': {
                    'dir': params_u['bc_values'][dir_u].reshape((3, -1), order='F') * ls, # [L]
                    'neu': params_u['bc_values'][neu_u].reshape((3, -1), order='F') * ss * ls**(dim-1), # [P L^(dim-1)]
                },
                'source':
                    params_u['source'] * ss * ls**(dim-1), # [P L^(dim-1)]
                'biot_alpha': # [ ]
                    params_u['biot_alpha'], 
                'mu':
                    params_u['fourth_order_tensor'].mu * ss, # [P]
                'lambda':
                    params_u['fourth_order_tensor'].lmbda * ss, # [P]
            }
        elif dim == s.Nd-1: # On fracture
            mech = params_u # We're only interested in the friction coefficient
        
        # On the fracture, terms are weighed by the specific volume!
        sv_scale = np.power(ls, s.Nd - dim)
        # FLOW
        flow = {
            'bc_values': {
                'dir': params_p['bc_values'][dir_p] * ss, # [P]
                'neu': params_p['bc_values'][neu_p] * ls**dim, # [L^(dim)]
            },
            'mass_weight':  
                params_p['mass_weight'] * sv_scale / ss ,  # [L^(Nd-dim) / P]
            'source': # -- integrated source is of dim Nd regardless of 
                      #    ambient dimension!!
                params_p['source'] * ls**s.Nd, # [L^Nd] 
            'biot_alpha': # [ ]
                params_p['biot_alpha'],
            'kxx':  # k/mu * aperture^(Nd-dim) : [L^(Nd-dim) * L^2 / P]
                params_p['second_order_tensor'].values[0, 0, :] * ls**2 / ss * sv_scale, 
        }
        return mech, flow
    
    mech1, flow1 = fetch_values_from_setup(s1, dim)
    mech2, flow2 = fetch_values_from_setup(s2, dim)
    
    # -- ASSERTIONS --
    # Mechanics
    if dim == s1.Nd:
        # BC:
        assert np.allclose(mech1['bc_values']['dir'], mech2['bc_values']['dir'])
        assert np.allclose(mech1['bc_values']['neu'], mech2['bc_values']['neu'])
        # Source
        assert np.allclose(mech1['source'], mech2['source'])
        # Physical parameters
        assert np.allclose(mech1['biot_alpha'], mech2['biot_alpha'])
        assert np.allclose(mech1['mu'], mech2['mu'])
        assert np.allclose(mech1['lambda'], mech2['lambda'])
    elif dim == s1.Nd-1:
        assert np.allclose(mech1['friction_coefficient'], mech2['friction_coefficient'])
    
    # Flow
    # BC:
    assert np.allclose(flow1['bc_values']['dir'], flow2['bc_values']['dir'])
    assert np.allclose(flow1['bc_values']['neu'], flow2['bc_values']['neu'])
    # Source
    assert np.allclose(flow1['source'], flow2['source'])
    # Physical parameters
    assert np.allclose(flow1['biot_alpha'], flow2['biot_alpha'])
    assert np.allclose(flow1['kxx'], flow2['kxx'])
    
def check_mortar_parameter_scaling(s1, s2):
    def fetch_values_from_setup(s):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        
        for e, d in gb.edges():
            k = d['parameters']['flow']['normal_diffusivity'] * ls/ss
        return k
    
    k1 = fetch_values_from_setup(s1)
    k2 = fetch_values_from_setup(s2)
    
    assert np.allclose(k1,k2)

In [5]:
def assert_solutions(s1, s2, dim):
    """ Assert that solutions scale correctly
    
    Parameters
    ----------
    s1, s2 : ContactMechanicsBiotISC
        Setups
    dim : int
        which grid dimension to extract solutions over
    """
    print(f"Checking solutions for two setups on grid of dimension: {dim}")
    
    def fetch_values_from_setup(s, dim):
        # scales
        ls = s.length_scale
        ss = s.scalar_scale
        # grids
        gb = s.gb
        g = gb.grids_of_dimension(dim)[0]
        # data
        d = gb.node_props(g)
        state = d[pp.STATE]
        
        ### Quantities ###
        cell_centers = g.cell_centers * ls
        cell_volumes = g.cell_volumes * (ls)**dim
        face_areas = g.face_areas * (ls)**(dim-1)
        p = state['p'] * ss
        
        Nd = s.Nd
        # Matrix displacement
        if dim == Nd:
            u = state['u'] * ls
            
            # These are in reality only fetched on fracture. 
            # Set to 0 if 3d-matrix is checked.
            tangential_traction = [0]
            normal_traction = [0]
            u_t_norm = [0]
            u_n = [0]
        elif dim == Nd - 1:
            # Fetch the mortar grid and data
            e, de = [i for i in gb.edges()][0]
            
            # Displacement (global coords)
            mg = de['mortar_grid']
            mortar_u = de['state']['mortar_u']
            u = (mg.mortar_to_slave_avg(nd=Nd) 
                 * mg.sign_of_mortar_sides(nd=Nd)
                 * mortar_u) * ls
            
            # Displacement (local coords)
            u_local = s.reconstruct_local_displacement_jump(de, from_iterate=False) *ls
            u_t_norm = np.linalg.norm(u_local[:2, :], axis=0)
            u_n = u_local[2, :]
            
            # Traction
            # recall: traction is represented in local coordinates directly.
            proj = de['tangential_normal_projection']
            proj.tangential_basis[:,:,0]
            traction = d['state']['contact_traction']
            tangential_traction_local = traction[:-1]
            normal_traction = traction[-1]
            tangential_traction = proj.tangential_basis[:,:,0].dot(tangential_traction_local)
            # upscale the scaled result
            normal_traction = normal_traction * ss * ls**(Nd-1)
            tangential_traction = tangential_traction * ss * ls**(Nd-1)

        
        return (cell_centers, cell_volumes, face_areas, p, u, 
                normal_traction, tangential_traction, u_t_norm, u_n)
    
    # Get the results
    (cell_centers1, cell_volumes1, face_areas1, p1, u1, 
     normal_traction1, tangential_traction1, u_t_norm1, u_n1
    ) = fetch_values_from_setup(s1, dim)
    
    (cell_centers2, cell_volumes2, face_areas2, p2, u2, 
     normal_traction2, tangential_traction2, u_t_norm2, u_n2
    ) = fetch_values_from_setup(s2, dim)

    ### Assertions ###
    # Assert that each geometric index corresponds to one another
    assert np.allclose(cell_centers1, cell_centers2)
    
    # Assert that all volumes and areas are the same
    assert np.allclose(cell_volumes1, cell_volumes2)
    assert np.allclose(face_areas1, face_areas2)
    
    ## Non-zero on all grids ##
    # Assert that the pressure results are equal
    p_true = np.allclose(p1, p2)
#     assert p_true
    print(f"p: {p_true}")
    
    # Assert that the displacement results are equal
    u_true = np.allclose(u1, u2)
#     assert u_true
    print(f"u: {u_true}")
    
    # Assert that norm of tangential displacement is equal
    u_t_norm_true = np.allclose(u_t_norm1, u_t_norm2)
    print(f"u_t_norm: {u_t_norm_true}")
    # Assert that norm displacement is equal
    u_n_true = np.allclose(u_n1, u_n2)
    print(f"u_n: {u_n_true}")


    # Traction
    n_traction_true = np.allclose(normal_traction1, normal_traction2)
#     assert n_traction_true
    print(f"normal_traction: {n_traction_true}")
    t_traction_true = np.allclose(tangential_traction1, tangential_traction2)
#     assert t_traction_true
    print(f"tangential_traction: {t_traction_true}")
    
    assert (p_true and u_true and n_traction_true and t_traction_true
            and u_t_norm_true and u_n_true
           )

## Unscaled setup

In [23]:
params={
    "length_scale": 1,
    "scalar_scale": 1,
}

un = SimpleContactMechanicsBiot(params)
pp.run_time_dependent_model(un, {"nl_convergence_tol": 1e-20})

INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 12 intersections. Ellapsed time: 0.01305
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00049
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 48 points, 36
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 8 points, 12 edges. Ellapsed time
                    0.01131
INFO:porepy.fracs.fractures:Remove edge intersections
INFO:porepy.fracs.fractures:Done with intersection removal. Elapsed time 0.02655
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 8 points, 12
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 8 points, 12 edges. Ellapsed time
                    0.00848
INFO:porepy.fracs.fractures:Finished fracture splitting after 0.06203 seconds
INFO:porepy.fracs.meshing:Assembl

# Scale by length

In [7]:
params={
    "length_scale": 10,
    "scalar_scale": 1,
}

ls10 = SimpleContactMechanicsBiot(params)
pp.run_time_dependent_model(ls10, {"nl_convergence_tol": 1e-20})

INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 12 intersections. Ellapsed time: 0.01303
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00052
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 52 points, 40
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed time
                    0.01184
INFO:porepy.fracs.fractures:Remove edge intersections
INFO:porepy.fracs.fractures:Done with intersection removal. Elapsed time 0.03298
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 12 points, 16
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed time
                    0.01104
INFO:porepy.fracs.fractures:Finished fracture splitting after 0.07304 seconds
INFO:porepy.fracs.meshing:Asse

# Check scaling of parameters and solutions

In [8]:
check_mortar_parameter_scaling(un, ls10)

In [9]:
check_parameter_scaling(un, ls10, 3)

check_parameter_scaling(un, ls10, 2)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2


In [10]:
assert_solutions(un, ls10, 3)

assert_solutions(un, ls10, 2)

Checking solutions for two setups on grid of dimension: 3
p: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True
Checking solutions for two setups on grid of dimension: 2
p: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


# More tests
## Scale in  scalar scale

In [24]:
# Just a weird looking scale to ensure no "cancellations" of length- and scalar scaling
params={
    "length_scale": 1,
    "scalar_scale": 10,
}

ss10 = SimpleContactMechanicsBiot(params)
pp.run_time_dependent_model(ss10, {"nl_convergence_tol": 1e-20})

INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 12 intersections. Ellapsed time: 0.01196
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00105
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 48 points, 36
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 8 points, 12 edges. Ellapsed time
                    0.01235
INFO:porepy.fracs.fractures:Remove edge intersections
INFO:porepy.fracs.fractures:Done with intersection removal. Elapsed time 0.02431
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 8 points, 12
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 8 points, 12 edges. Ellapsed time
                    0.01473
INFO:porepy.fracs.fractures:Finished fracture splitting after 0.06524 seconds
INFO:porepy.fracs.meshing:Assembl

In [25]:
check_parameter_scaling(ss10, un, 3)
check_parameter_scaling(ss10, un, 2)
check_mortar_parameter_scaling(ss10, un)

Checking parameters for two setups on grid of dimension: 3


In [26]:
assert_solutions(ss10, un, 3)

Checking solutions for two setups on grid of dimension: 3
p: False
u: False
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


AssertionError: 

In [27]:
%debug

> <ipython-input-5-3f2891dc12cb>(118)assert_solutions()
    115     print(f"tangential_traction: {t_traction_true}")
    116 
    117     assert (p_true and u_true and n_traction_true and t_traction_true
--> 118             and u_t_norm_true and u_n_true
    119            )

ipdb> p1-p2
array([0.21515783, 0.0901467 , 0.34689369, 0.17474784, 0.34245372,
       0.17918782, 0.20758288, 0.09772165, 0.29643089, 0.13743391,
       0.49591328, 0.26388869, 0.49370628, 0.26609569, 0.29431474,
       0.13955006, 0.29248476, 0.1377875 , 0.48487904, 0.25961052,
       0.48408391, 0.26040566, 0.29278628, 0.13748598, 0.18781731,
       0.07549589, 0.29332901, 0.13826297, 0.29243182, 0.13916017,
       0.18743151, 0.07588168])
ipdb> q


In [15]:
assert_solutions(ss10, un, 2)

Checking solutions for two setups on grid of dimension: 2
p: False
u: False
u_t_norm: False
u_n: True
normal_traction: False
tangential_traction: False


AssertionError: 

In [16]:
%debug

> <ipython-input-5-3f2891dc12cb>(118)assert_solutions()
    115     print(f"tangential_traction: {t_traction_true}")
    116 
    117     assert (p_true and u_true and n_traction_true and t_traction_true
--> 118             and u_t_norm_true and u_n_true
    119            )

ipdb> u_n1
array([0.])
ipdb> p1-p2
array([0.41181413])
ipdb> p1
array([0.72812296])
ipdb> p2
array([0.31630883])
ipdb> u_t_norm1
array([1.23493366])
ipdb> u_t_norm2
array([0.13267313])
ipdb> normal_traction1
-1.9426735120579455
ipdb> normal_traction2
-4.177067554853968
ipdb> (normal_traction1-normal_traction2)/normal_traction2
-0.5349192976779946
ipdb> q


# Another test

In [17]:
# Just a weird looking scale to ensure no "cancellations" of length- and scalar scaling
params={
    "length_scale": 13,
    "scalar_scale": 1,
}

hard = SimpleContactMechanicsBiot(params)
pp.run_time_dependent_model(hard, {"nl_convergence_tol": 1e-20})

INFO:porepy.fracs.fractures:Find intersection between fratures
INFO:porepy.fracs.fractures:Found 12 intersections. Ellapsed time: 0.01338
INFO:porepy.fracs.fractures:Split intersections
INFO:porepy.fracs.fractures:Compile list of points and edges
INFO:porepy.fracs.fractures:Points and edges done. Elapsed time 0.00088
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 52 points, 40
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed time
                    0.01180
INFO:porepy.fracs.fractures:Remove edge intersections
INFO:porepy.fracs.fractures:Done with intersection removal. Elapsed time 0.03051
INFO:porepy.fracs.fractures:Uniquify points and edges, starting with 12 points, 16
                    edges
INFO:porepy.fracs.fractures:Uniquify complete. 12 points, 16 edges. Ellapsed time
                    0.00931
INFO:porepy.fracs.fractures:Finished fracture splitting after 0.06770 seconds
INFO:porepy.fracs.meshing:Asse

In [18]:
check_parameter_scaling(hard, un, 3)
check_parameter_scaling(hard, un, 2)
check_mortar_parameter_scaling(hard, un)

Checking parameters for two setups on grid of dimension: 3
Checking parameters for two setups on grid of dimension: 2


In [19]:
assert_solutions(hard, un, 3)

Checking solutions for two setups on grid of dimension: 3
p: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


In [ ]:
%debug

In [20]:
assert_solutions(hard, un, 2)

Checking solutions for two setups on grid of dimension: 2
p: True
u: True
u_t_norm: True
u_n: True
normal_traction: True
tangential_traction: True


In [ ]:
%debug

# Test two moderately scaled solutions

In [ ]:
# Just a weird looking scale to ensure no "cancellations" of length- and scalar scaling
params={
    "length_scale": 10,
    "scalar_scale": 5,
}

t1 = SimpleContactMechanicsBiot(params)
pp.run_time_dependent_model(t1, {"nl_convergence_tol": 1e-30})

In [ ]:
# Just a weird looking scale to ensure no "cancellations" of length- and scalar scaling
params={
    "length_scale": 20,
    "scalar_scale": 10,
}

t2 = SimpleContactMechanicsBiot(params)
pp.run_time_dependent_model(t2, {"nl_convergence_tol": 1e-30})

In [ ]:
check_parameter_scaling(t1, t2, 3)
check_parameter_scaling(t1, t2, 2)
check_mortar_parameter_scaling(t1, t2)

In [ ]:
assert_solutions(t1, t2, 3)

In [ ]:
%debug

In [ ]:
assert_solutions(t1, t2, 2)

In [ ]:
%debug